# CSCI 3155 : Homework 5

Evaluating expressions

__Name__:

Kyle Moe

In [23]:
// TEST HELPER: Do not EDIT this cell please.
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined function passed

## Artithmetic Expressions

$$\begin{array}{rcll}
\mathbf{AExpr} & := & Const(Int) \\
 & | & Ident(String) \\
 & | & Plus(\mathbf{AExpr}, \mathbf{AExpr}) \\
 & | & Minus(\mathbf{AExpr}, \mathbf{AExpr}) \\
 & | & Mult (\mathbf{AExpr}, \mathbf{AExpr}) \\
 & | & Neg (\mathbf{AExpr}) \\
\end{array}$$



$$ eval : AExpr \rightarrow Env \rightarrow Int $$

In [24]:
sealed trait AExpr
case class Const(n : Int) extends AExpr
case class Ident(s : String) extends AExpr
case class Plus( e1 : AExpr, e2 : AExpr) extends AExpr
case class Minus( e1 : AExpr, e2 : AExpr) extends AExpr
case class Mult( e1 : AExpr, e2 : AExpr) extends AExpr
case class Neg( e1 : AExpr) extends AExpr

def evalA( e : AExpr, env : Map[String,Int]) : Int = 
    e match {
        case Const(n)      => n 
        case Ident(s)      => env(s)
        case Plus(e1, e2)  => evalA(e1,env) + evalA(e2,env)
        case Minus(e1, e2) => evalA(e1,env) - evalA(e2,env)
        case Mult(e1, e2)  => evalA(e1,env) * evalA(e2,env)
        case Neg(e1)       => -evalA(e1,env)
    }

val myAExpr = Plus( Const(3), 
                   Mult( 
                       Ident("x"), 
                       Const(5)))
val myenv = Map("x" -> 4)
evalA( myAExpr, myenv)

defined trait AExpr
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Neg
defined function evalA
myAExpr: Plus = Plus(Const(3), Mult(Ident("x"), Const(5)))
myenv: Map[String, Int] = Map("x" -> 4)
res23_10: Int = 23

## Problem 1 : Boolean expressions

### Problem 1a (10 points)
Write a new type for boolean expressions, **BExpr**. This type includes everything necessary for simple boolean expressions: the boolean constants True and False, the boolean negation operation Not, the boolean binary operators And and Or, as well as ITE, the if then else expression. 

$$\begin{array}{rcll}
\mathbf{BExpr} & := & True \\
 & | & False \\
 & | & Not ( \mathbf{BExpr} )\\
 & | & And ( \mathbf{BExpr}, \mathbf{BExpr} ) \\
 & | & Or ( \mathbf{BExpr}, \mathbf{BExpr} ) \\
 & | & ITE(\mathbf{BExpr}, \mathbf{BExpr}, \mathbf{BExpr})\\
\end{array}$$

In [25]:
// Type def for Boolean Expression with True, False Base cases
sealed trait BExpr
case object True extends BExpr
case object False extends BExpr
case class Not(e : BExpr) extends BExpr
case class And(e1 : BExpr, e2 : BExpr) extends BExpr
case class Or(e1 : BExpr, e2 : BExpr) extends BExpr
case class ITE(e1 : BExpr, e2 : BExpr, e3 : BExpr) extends BExpr


defined trait BExpr
defined object True
defined object False
defined class Not
defined class And
defined class Or
defined class ITE

In [26]:
val bexpr1 : BExpr = True
val bexpr2 : BExpr = Not(bexpr1)
val bexpr3 : BExpr = And(Not(bexpr1), True)
val bexpr4 : BExpr = Or(Not(bexpr3), bexpr3)
val bexpr5 : BExpr = ITE(bexpr3, False, True)
val bexpr6 : BExpr = ITE(bexpr4, bexpr3, bexpr5)
passed(10)


*** Tests Passed (10 points) ***


bexpr1: BExpr = True
bexpr2: BExpr = Not(True)
bexpr3: BExpr = And(Not(True), True)
bexpr4: BExpr = Or(Not(And(Not(True), True)), And(Not(True), True))
bexpr5: BExpr = ITE(And(Not(True), True), False, True)
bexpr6: BExpr = ITE(
  Or(Not(And(Not(True), True)), And(Not(True), True)),
  And(Not(True), True),
  ITE(And(Not(True), True), False, True)
)

### Problem 1b (20 points)
Write an evaluation function for boolean expressions using the above type.

$$\texttt{evalB} : BExpr\rightarrow Boolean$$

In [27]:
// Evaluates a boolean expression starting with base case checks
def evalB( e : BExpr) : Boolean = {
    e match {
        case True => true 
        case False => false
        case Not(p) => !evalB(p) // Not of a boolean recursion
        case And(p1, p2) => evalB(p1) && evalB(p2) // And checks each 'lower branch' statements
        case Or(p1, p2) => evalB(p1) || evalB(p2) // Or like And above
        case ITE(p1, p2, p3) => if (evalB(p1)) evalB(p2) else evalB(p3) // ITE using inline if else and recursion
    }
}

defined function evalB

In [28]:
assert(evalB(bexpr1) == true,  "test 1")
assert(evalB(bexpr2) == false, "test 2")
assert(evalB(bexpr3) == false, "test 3")
assert(evalB(bexpr4) == true,  "test 4")
assert(evalB(bexpr5) == true, " test 5")
assert(evalB(bexpr6) == false, "test 6")
passed(20)


*** Tests Passed (20 points) ***


## Problem 2: Combined expressions too.

### Problem 1a (10 points)

Write a new type **CExpr** for mixed arithmetic and boolean expressions. The new expressions should include everything from before, including the necessary components for both arithmetic boolean expressions, as well as the parts of useful combined expressions, Geq for numerical comparison, Eq for equality.


$$\begin{array}{rcll}
\mathbf{CExpr} & := & Const(Int) \\
 & | & True \\
 & | & False \\
 & | & Ident(String) \\
 & | & Plus(\mathbf{CExpr}, \mathbf{CExpr}) \\
 & | & Minus(\mathbf{CExpr}, \mathbf{CExpr}) \\
 & | & Mult (\mathbf{CExpr}, \mathbf{CExpr}) \\
 & | & Neg (\mathbf{CExpr}) \\
 & | & And ( \mathbf{CExpr}, \mathbf{CExpr} ) \\
 & | & Or ( \mathbf{CExpr}, \mathbf{CExpr} ) \\
 & | & Geq (\mathbf{CExpr}, \mathbf{CExpr}) \\
 & | & Eq (\mathbf{CExpr}, \mathbf{CExpr}) \\
 & | & ITE(\mathbf{CExpr}, \mathbf{CExpr}, \mathbf{CExpr})\\
\end{array}$$

NOTE: We will be reusing the constructor names in this notebook. Make sure to run the cell with the definitions you are using for each test!

In [29]:
// Type def for General Expression in the order above
sealed trait CExpr
case class Const(n : Int) extends CExpr
case object True extends CExpr
case object False extends CExpr
case class Ident(s : String) extends CExpr
case class Plus( e1 : CExpr, e2 : CExpr) extends CExpr
case class Minus( e1 : CExpr, e2 : CExpr) extends CExpr
case class Mult( e1 : CExpr, e2 : CExpr) extends CExpr
case class Neg( e1 : CExpr) extends CExpr
case class And(e1 : CExpr, e2 : CExpr) extends CExpr
case class Not(e : CExpr) extends CExpr
case class Or(e1 : CExpr, e2 : CExpr) extends CExpr
case class Geq(e1 : CExpr, e2 : CExpr) extends CExpr
case class Eq(e1 : CExpr, e2 : CExpr) extends CExpr
case class ITE(e1 : CExpr, e2 : CExpr, e3 : CExpr) extends CExpr

defined trait CExpr
defined class Const
defined object True
defined object False
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Neg
defined class And
defined class Not
defined class Or
defined class Geq
defined class Eq
defined class ITE

In [30]:
val expr1 : CExpr = Const(10)
val expr2 : CExpr = True
val expr3 : CExpr = False
val expr4 : CExpr = Ident("x")
val expr5 : CExpr = Plus(expr1, expr4)
val expr6 : CExpr = Minus(expr5, expr1)
val expr7 : CExpr = Mult(expr5, expr6)
val expr8 : CExpr = Neg(expr7)
val expr9 : CExpr = And(expr2,expr3)
val expr10 : CExpr = Or(expr2, expr9)
val expr11 : CExpr = Geq(expr1, expr4)
val expr12 : CExpr = Eq(expr1, expr4) 
val expr13 : CExpr = ITE(expr2, expr1, expr4)
passed(10)


*** Tests Passed (10 points) ***


expr1: CExpr = Const(10)
expr2: CExpr = True
expr3: CExpr = False
expr4: CExpr = Ident("x")
expr5: CExpr = Plus(Const(10), Ident("x"))
expr6: CExpr = Minus(Plus(Const(10), Ident("x")), Const(10))
expr7: CExpr = Mult(
  Plus(Const(10), Ident("x")),
  Minus(Plus(Const(10), Ident("x")), Const(10))
)
expr8: CExpr = Neg(
  Mult(
    Plus(Const(10), Ident("x")),
    Minus(Plus(Const(10), Ident("x")), Const(10))
  )
)
expr9: CExpr = And(True, False)
expr10: CExpr = Or(True, And(True, False))
expr11: CExpr = Geq(Const(10), Ident("x"))
expr12: CExpr = Eq(Const(10), Ident("x"))
expr13: CExpr = ITE(True, Const(10), Ident("x"))

### Problem 2b (10 points)

Our eval function is no longer going to be able to return a single type since the expressions can return either Integers or Booleans. However we still want to be able to return a value for each expression. We might want to return either a number value or NumVal for integer or a boolean value or BinVal for booleans. If we run into an error in our expression we will want to return an Error.

$$\begin{array}{rcll}
\mathbf{Value} & := & NumVal(Int) \\
 & | & BinVal(Boolean) \\
 & | & Error \\
\end{array}$$

Write the **Value** type in scala:

In [31]:
// Value type with Num and Bin constructors and Error object
sealed trait Value
case class NumVal(i : Int) extends Value
case class BinVal(b : Boolean) extends Value
case object Error extends Value

defined trait Value
defined class NumVal
defined class BinVal
defined object Error

In [32]:
val a : Value = NumVal(10)
val b : Value = NumVal(-10)
val c : Value = BinVal(true)
val d : Value = Error
passed(10)


*** Tests Passed (10 points) ***


a: Value = NumVal(10)
b: Value = NumVal(-10)
c: Value = BinVal(true)
d: Value = Error

### Problem 2c (10 points)

Environments store what values different variables represent. For arithmetic expressions the environment we used mapped variable names to integers. However in our new combined expressions integers aren't the only things that can be stored in the environment. What should the type of the environment be?

Scala allows us to define type aliases with the following syntax:

```type <Type Alias Name> = <Type Definition>```

If we wanted to introduce a new type in scala for bitstrings we could instead create a type alias for lists of booleans:

```type BitString = List[Boolean]```


What is the type of the environment for combined expressions?

In [33]:
// We want our type Env to represent a map from a string (variable) to a Value obj
type Environment = Map[String, Value]

defined type Environment

In [34]:
val emptyenv : Environment = Map()
val env1 : Environment = Map( "a" -> NumVal(10))
val env2 : Environment = Map( "b" -> BinVal(true))
val env3 : Environment = Map( "x" -> NumVal(3), "y" -> NumVal(4), "z" -> BinVal(true))
passed(10)


*** Tests Passed (10 points) ***


emptyenv: Environment = Map()
env1: Environment = Map("a" -> NumVal(10))
env2: Environment = Map("b" -> BinVal(true))
env3: Environment = Map("x" -> NumVal(3), "y" -> NumVal(4), "z" -> BinVal(true))

### Problem 2d (20 points)

Write ``evalC``, the interpreter for evaluating combined arithmetic/boolean expressions:

$$ evalC : CExpr \rightarrow Environment \rightarrow Value $$

Suggestion: Start with one case at a time, and always consider the types of what being returned by every function.

In [35]:
// Use pattern matching for evalC to distinguish the cases of the entered expression
def evalC( e : CExpr, env : Environment) : Value = {
    e match {
        case Const(i) => NumVal(i) // Handle constant int base case (return Value)
        case True => BinVal(true) // Base cases for BinVal
        case False => BinVal(false)
        case Ident(s) => if (env.contains(s)) env(s) else Error // Return env variable if exists otherwise Error
        
        // Pattern match the tuple of inputs to check that both are NatVals and add, othewise Error
        case Plus(p1, p2) => val e1 : Value = evalC(p1, env)
                             val e2 : Value = evalC(p2, env)
                             (e1, e2) match {
                                 case (NumVal(e1), NumVal(e2)) => NumVal(e1+e2)
                                 case _ => Error
                             }
        // Likewise with Minus and Mult, pattern match for Ints and return Value
        case Minus(p1, p2) => val e1 : Value = evalC(p1, env)
                              val e2 : Value = evalC(p2, env)
                              (e1, e2) match {
                                  case (NumVal(e1), NumVal(e2)) => NumVal(e1-e2)
                                  case _ => Error
                              }
        case Mult(p1, p2) => val e1 : Value = evalC(p1, env)
                             val e2 : Value = evalC(p2, env)
                             (e1, e2) match {
                                 case (NumVal(e1), NumVal(e2)) => NumVal(e1*e2)
                                 case _ => Error
                             }
        // Negate both Boolean and Int Values 
        case Neg(p) => val e : Value = evalC(p, env)
                       e match {
                           case NumVal(i) => NumVal(-i)
                           case BinVal(b) => BinVal(!b)
                           case _ => Error
                       }
        // With And, Or, and Geq run pattern matching to check boolean type and return BinVal of the conditional
        case And(b1, b2) => val e1 : Value = evalC(b1, env)
                            val e2 : Value = evalC(b2, env)
                            (e1, e2) match {
                                case (BinVal(p1), BinVal(p2)) => BinVal(p1 && p2)
                                case _ => Error
                            }
        case Or(b1, b2) => val e1 : Value = evalC(b1, env)
                           val e2 : Value = evalC(b2, env)
                           (e1, e2) match {
                               case (BinVal(e1), BinVal(e2)) => BinVal(e1 || e2)
                               case _ => Error
                           }
        case Geq(p1, p2) => val e1 : Value = evalC(p1, env)
                            val e2 : Value = evalC(p2, env)
                            (e1, e2) match {
                                case (NumVal(e1), NumVal(e2)) => BinVal(e1 >= e2)
                                case _ => Error
                            }
        // For Eq we want to return a Bool if inputs are Values but only true if 
        // inputs are of the same Value type and equivalent
        case Eq(p1, p2) => val e1 : Value = evalC(p1, env)
                           val e2 : Value = evalC(p2, env)
                           (e1, e2) match {
                               case (NumVal(e1), NumVal(e2)) => BinVal(e1 == e2)
                               case (BinVal(e1), BinVal(e2)) => BinVal(e1 == e2)
                               case (BinVal(e1), NumVal(e2)) => BinVal(false)
                               case (NumVal(e1), BinVal(e2)) => BinVal(false)
                               case _ => Error
                           }
        // ITE performs a simple boolean pattern match and runs designated input
        case ITE(p1, p2, p3) => val e1 : Value = evalC(p1, env)
                                e1 match {
                                    case BinVal(true) => evalC(p2, env)
                                    case BinVal(false) => evalC(p3, env)
                                    case _ => Error
                                }
    }
}

defined function evalC

In [36]:
val e1 : CExpr = Const(10)
val e2 : CExpr = True
val emptyenv : Environment = Map()
assert(evalC( e1, emptyenv) == NumVal(10) , "test 1")
assert(evalC( e2, emptyenv) == BinVal(true), "test 2")
passed(4)


*** Tests Passed (4 points) ***


e1: CExpr = Const(10)
e2: CExpr = True
emptyenv: Environment = Map()

In [37]:
val e1 : CExpr = Ident("x")
val e2 : CExpr = Ident("y")
val e3 : CExpr = Ident("z")
val env : Environment = Map("x"-> NumVal(10), "y" -> BinVal(true))
val emptyenv : Environment = Map()
assert(evalC( e1, env) == NumVal(10), "test 3")
assert(evalC( e2, env) == BinVal(true), "test 4")
assert(evalC( e1, emptyenv) == Error)
assert(evalC( e3, env) == Error)
passed(4)


*** Tests Passed (4 points) ***


e1: CExpr = Ident("x")
e2: CExpr = Ident("y")
e3: CExpr = Ident("z")
env: Environment = Map("x" -> NumVal(10), "y" -> BinVal(true))
emptyenv: Environment = Map()

In [38]:
val e1 : CExpr = Plus(Const(3), Mult(Ident("x"), Const(5)))
val e2 : CExpr = Minus(Mult(Ident("x"), Ident("x")), Mult(Ident("y"), Ident("y")))
val env1 : Environment = Map("x"-> NumVal(4), "y" -> NumVal(5))
val env2 : Environment = Map("x"-> BinVal(true))
assert(evalC( e1, env1) == NumVal(23), "test 5")
assert(evalC( e1, env2) == Error, "test 6")
assert(evalC( e2, env1) == NumVal(-9), "test 7")
passed(4)


*** Tests Passed (4 points) ***


e1: CExpr = Plus(Const(3), Mult(Ident("x"), Const(5)))
e2: CExpr = Minus(Mult(Ident("x"), Ident("x")), Mult(Ident("y"), Ident("y")))
env1: Environment = Map("x" -> NumVal(4), "y" -> NumVal(5))
env2: Environment = Map("x" -> BinVal(true))

In [39]:
val e1 : CExpr = Neg(Const(10))
val e2 : CExpr = Neg(True)
val emptyenv : Environment = Map()
assert( evalC(e1, emptyenv) == NumVal(-10), "test 8")
assert( evalC(e2, emptyenv) == BinVal(false), "test 9")
passed(4)


*** Tests Passed (4 points) ***


e1: CExpr = Neg(Const(10))
e2: CExpr = Neg(True)
emptyenv: Environment = Map()

In [40]:
val e1 : CExpr = And(True, Or(Ident("x"), Neg(Ident("x"))))
val env : Environment = Map( "x" -> BinVal(false))
assert( evalC(e1, env) == BinVal(true), "test 10")
passed(4)


*** Tests Passed (4 points) ***


e1: CExpr = And(True, Or(Ident("x"), Neg(Ident("x"))))
env: Environment = Map("x" -> BinVal(false))

In [41]:
val e1 : CExpr = Geq(Const(10), Const(5))
val e2 : CExpr = Geq(Const(5), Const(10))
val e3 : CExpr = Geq(Const(5), Const(5))
val e4 : CExpr = Geq(True, Const(5))
val emptyenv : Environment = Map()
assert( evalC(e1, emptyenv) == BinVal(true), "test 11")
assert( evalC(e2, emptyenv) == BinVal(false), "test 12")
assert( evalC(e3, emptyenv) == BinVal(true), "test 13")
assert( evalC(e4, emptyenv) == Error, "test 14")
passed(4)


*** Tests Passed (4 points) ***


e1: CExpr = Geq(Const(10), Const(5))
e2: CExpr = Geq(Const(5), Const(10))
e3: CExpr = Geq(Const(5), Const(5))
e4: CExpr = Geq(True, Const(5))
emptyenv: Environment = Map()

In [42]:
val e1 : CExpr = Eq( Const(10), Ident("x"))
val e2 : CExpr = Plus( Const(10), True)
val env1 : Environment = Map( "x" -> NumVal(10))
val env2 : Environment = Map( "x" -> NumVal(5))
val env3 : Environment = Map( "x" -> BinVal(true))
assert( evalC(e1, env1) == BinVal(true), "test 15")
assert( evalC(e1, env2) == BinVal(false), "test 16")
assert( evalC(e1, env3) == BinVal(false), "test 17")
assert( evalC(e2, env1) == Error, "test 18")
passed(4)


*** Tests Passed (4 points) ***


e1: CExpr = Eq(Const(10), Ident("x"))
e2: CExpr = Plus(Const(10), True)
env1: Environment = Map("x" -> NumVal(10))
env2: Environment = Map("x" -> NumVal(5))
env3: Environment = Map("x" -> BinVal(true))

In [43]:
val e1 : CExpr = ITE( Geq(Ident("x"), Ident("y")), Ident("x"), Ident("y"))
val env1 : Environment = Map( "x" -> NumVal(10), "y" -> NumVal(20))
val env2 : Environment = Map( "x" -> NumVal(5), "y" -> NumVal(4))
assert( evalC(e1, env1) == NumVal(20), "test 19")
assert( evalC(e1, env2) == NumVal(5), "test 20")
passed(4)


*** Tests Passed (4 points) ***


e1: CExpr = ITE(Geq(Ident("x"), Ident("y")), Ident("x"), Ident("y"))
env1: Environment = Map("x" -> NumVal(10), "y" -> NumVal(20))
env2: Environment = Map("x" -> NumVal(5), "y" -> NumVal(4))

In [44]:
val e1 : CExpr =
    Mult( Ident("y"), 
        ITE( And( Geq(Ident("x"), Const(1)), Neg( Eq(Ident("x"), Const(0)))),
             Mult( 
                Minus( Ident("x"), Neg(Const(-1))), 
                Const(6)),
             Const(1))
        )
val env1 : Environment = Map("x" -> NumVal(5), "y" -> NumVal(5))
assert( evalC(e1, env1) == NumVal(120), "test 21")
passed(8)


*** Tests Passed (8 points) ***


e1: CExpr = Mult(
  Ident("y"),
  ITE(
    And(Geq(Ident("x"), Const(1)), Neg(Eq(Ident("x"), Const(0)))),
    Mult(Minus(Ident("x"), Neg(Const(-1))), Const(6)),
    Const(1)
  )
)
env1: Environment = Map("x" -> NumVal(5), "y" -> NumVal(5))